# W2M5: Sentiment Analysis

필요한 모듈을 설치합니다.

In [7]:
! pip install pandas
! pip install matplotlib
! pip install wordcloud

이하는 Sentiment140 dataset을 다운로드하고 압축을 푸는 코드입니다. 이미 csv파일이 있다면 실행하지 말아주세요.

In [2]:
! pwd
! curl -L -o ./sentiment140.zip\
  https://www.kaggle.com/api/v1/datasets/download/kazanova/sentiment140
! unzip sentiment140.zip

/Users/admin/Documents/GitHub/DataEngineering/W2/M5
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 80.9M  100 80.9M    0     0  11.7M      0  0:00:06  0:00:06 --:--:-- 13.4M
Archive:  sentiment140.zip
replace training.1600000.processed.noemoticon.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


프로그램을 작동하는데 필요한 모듈을 가져옵니다.

In [4]:
import pandas as pd
import multiprocessing as mp
import matplotlib.pyplot as plt
import wordcloud
import tqdm
import nltk

다음은 멀티 프로세싱을 위해 쓰레드 수를 설정하고 큐를 선언합니다.

In [ ]:
# constants
EXTRACT_TASK_COUNT = 4
TRANSFORM_TASK_COUNT = 8
LOAD_TASK_COUNT = 4

CSV_DIR = "training.1600000.processed.noemoticon.csv"
BATCH_SIZE = 1000
WHOLE_SIZE = 1600000

extractedQueuePos = mp.Queue()
extractedQueueNeg = mp.Queue()
transformedQueue = mp.Queue()


다음은 다운로드 받은 csv파일을 읽는 코드입니다.

In [ ]:
def extract(sem, donePos, doneNeg):
    startLine = sem.get()
    while startLine < WHOLE_SIZE:
        data = pd.read_csv("training.1600000.processed.noemoticon.csv",
                           encoding='ISO-8859-1',
                           names=["target", "id","date", "flag", "user", "text" ],
                           skiprows=startLine,
                           chunksize=BATCH_SIZE
                          )
        if not data[data["target"] == 0].empty:
            doneNeg.put(data[data["target"] == 0])
        if not data[data["target"] == 4].empty:
            donePos.put(data[data["target"] == 4])
        sem.put(BATCH_SIZE * EXTRACT_TASK_COUNT + startLine)
        startLine = sem.get()
    sem.put(startLine)

다음은 각 문장을 token화 하는 Transform 함수입니다.

In [ ]:
def transformTokinize(df):
    stop_words = set(nltk.corpus.stopwords.words('english'))
    lmtzr = nltk.stem.WordNetLemmatizer()
    stemmer = nltk.stem.PorterStemmer()
    gTokens = []
    for sentence in tqdm.tqdm(df, desc=f"{mp.current_process().name}: tokenizing"):
        target = sentence["target"]
        sentence = sentence["text"]
        tokens = [t for t in nltk.word_tokenize(sentence)]
        tokens = [t for t in tokens if t not in stop_words]
        tokens = [t for t in tokens if len(t) >= 3]
        tokens = [t.lower() for t in tokens]
        tokens = [lmtzr.lemmatize(t, 'v') for t in tokens]
        tokens = [stemmer.stem(t) for t in tokens]
        [gTokens.append(t) for t in tokens]
    data = pd.Series(gTokens)
    return data

In [7]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv",
                    encoding='ISO-8859-1',
                    names=["target", "id","date", "flag", "user", "text" ]
                    )
print("data count = ", data.shape[0])


data count =  1600000
